In [2]:
import pandas as pd

data_Audi = pd.read_csv("Audi_Desc_Count.csv")

In [3]:
comments_Audi = data_Audi[data_Audi['body'].notna()]
comments_Audi = comments_Audi.reset_index()
comments_Audi = comments_Audi.drop(columns=['index'])
# comments_Audi

In [4]:
df = comments_Audi.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keert\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk import PorterStemmer
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

#Stemming - words are reduced to their root form
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['it', 'was', '250k', 'including', 'tax', 'but', 'it', 'was', 'worth', 'it.', '222', 'made']


 tokenized and lemmatized document: 
['includ', 'worth']


In [8]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 carbon
1 chrome
2 definit
3 delet
4 fiber
5 mirror
6 diffus
7 rear
8 splitter
9 case
10 exhaust


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [12]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.071*"look" + 0.035*"delet" + 0.033*"wheel" + 0.026*"like" + 0.026*"black" + 0.023*"love" + 0.023*"nice" + 0.019*"color" + 0.015*"beauti" + 0.015*"grill"
Topic: 1 
Words: 0.023*"engin" + 0.018*"issu" + 0.015*"time" + 0.014*"tune" + 0.012*"chang" + 0.010*"replac" + 0.010*"like" + 0.009*"mile" + 0.009*"turbo" + 0.008*"drive"
Topic: 2 
Words: 0.057*"audi" + 0.056*"http" + 0.018*"post" + 0.016*"model" + 0.014*"imgur" + 0.010*"reddit" + 0.009*"comment" + 0.008*"auto" + 0.008*"video" + 0.008*"link"
Topic: 3 
Words: 0.028*"like" + 0.025*"drive" + 0.020*"look" + 0.020*"car" + 0.017*"audi" + 0.014*"think" + 0.014*"year" + 0.012*"want" + 0.012*"love" + 0.010*"better"
Topic: 4 
Words: 0.027*"thank" + 0.018*"audi" + 0.014*"work" + 0.013*"know" + 0.012*"tire" + 0.011*"dealer" + 0.011*"warranti" + 0.010*"go" + 0.010*"good" + 0.008*"buy"


In [13]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.078493  0.054941       1        1  27.972290
4     -0.010938  0.171443       2        1  23.896223
1      0.056873  0.175210       3        1  22.411499
0      0.229720 -0.264830       4        1  14.092572
2     -0.354147 -0.136764       5        1  11.627418, topic_info=     Category          Freq    Term         Total  loglift  logprob
30    Default  15949.000000    look  15949.000000  30.0000  30.0000
460   Default   6611.000000    http   6611.000000  29.0000  29.0000
35    Default  15913.000000    audi  15913.000000  28.0000  28.0000
140   Default   6575.000000   thank   6575.000000  27.0000  27.0000
3     Default   5241.000000   delet   5241.000000  26.0000  26.0000
...       ...           ...     ...           ...      ...      ...
453    Topic5    692.550232    edit   1399.084717   1.4486  -5.1295
905    Topic5    624.873474   avant   1577.778564   1.2256  -5.2324
1049   Topic5    550.838501   avail    877.176514   1.6865  -5.3585
112    Topic5    633.664246  option   2333.801758   0.8481  -5.2184
565    Topic5    552.682312  design   1343.997803   1.2632  -5.3551

[301 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1084      2  0.996598   accid
34        2  0.998530   advic
199       1  0.999293  afford
629       1  0.826222    agre
629       4  0.173461    agre
...     ...       ...     ...
138       3  0.097140    year
138       5  0.032947    year
724       4  0.998037  yellow
1554      5  0.998912   youtu
1447      5  0.997894  youtub

[419 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 3])

In [14]:
import pandas as pd

data_BMW = pd.read_csv("BMW_Desc_Count.csv")

In [15]:
comments_BMW = data_BMW[data_BMW['body'].notna()]
comments_BMW = comments_BMW.reset_index()
comments_BMW = comments_BMW.drop(columns=['index'])

In [16]:
df = comments_BMW.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [17]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['i’m', 'looking', 'for', 'one', 'of', 'these']


 tokenized and lemmatized document: 
['look']


In [18]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 delet
1 dealership
2 discount
3 forum
4 gener
5 guy
6 lightli
7 look
8 model
9 one
10 purchas


In [19]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [22]:
lda_model_BMW = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model_BMW.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"drive" + 0.024*"like" + 0.018*"car" + 0.016*"think" + 0.014*"seri" + 0.012*"peopl" + 0.012*"feel" + 0.011*"differ" + 0.010*"sport" + 0.010*"better"
Topic: 1 
Words: 0.069*"http" + 0.050*"comment" + 0.047*"post" + 0.046*"reddit" + 0.043*"messag" + 0.034*"review" + 0.030*"manual" + 0.027*"moder" + 0.023*"compos" + 0.022*"submiss"
Topic: 2 
Words: 0.027*"thank" + 0.016*"tire" + 0.011*"drive" + 0.011*"know" + 0.010*"go" + 0.010*"time" + 0.009*"work" + 0.008*"snow" + 0.008*"right" + 0.008*"think"
Topic: 3 
Words: 0.021*"year" + 0.015*"engin" + 0.014*"mile" + 0.010*"replac" + 0.010*"issu" + 0.009*"time" + 0.009*"buy" + 0.009*"good" + 0.009*"like" + 0.009*"go"
Topic: 4 
Words: 0.097*"look" + 0.038*"delet" + 0.036*"like" + 0.033*"nice" + 0.030*"love" + 0.028*"wheel" + 0.019*"color" + 0.017*"grill" + 0.017*"beauti" + 0.016*"black"


In [23]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_BMW, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.165472  0.124674       1        1  25.389633
0      0.126386  0.004923       2        1  25.169212
2      0.155792  0.139222       3        1  19.619225
1     -0.453417  0.090142       4        1  15.446597
4      0.005767 -0.358960       5        1  14.375335, topic_info=     Category          Freq     Term         Total  loglift  logprob
7     Default  26138.000000     look  26138.000000  30.0000  30.0000
210   Default  18143.000000     http  18143.000000  29.0000  29.0000
302   Default  13035.000000  comment  13035.000000  28.0000  28.0000
323   Default  12369.000000     post  12369.000000  27.0000  27.0000
328   Default  12077.000000   reddit  12077.000000  26.0000  26.0000
...       ...           ...      ...           ...      ...      ...
148    Topic5   3623.128418     good  10250.909180   0.8996  -4.2072
1039   Topic5   1528.599976     amaz   2308.979004   1.5272  -5.0702
56     Topic5   2839.882080    think  14283.819336   0.3243  -4.4508
11     Topic5   1328.187134      see   3405.345703   0.9981  -5.2107
67     Topic5   1251.204956     rear   2983.481445   1.0707  -5.2704

[276 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
298       4  0.999900  account
299       4  0.999756   action
436       2  0.999624     agre
2137      5  0.999511   alpina
1039      2  0.337812     amaz
...     ...       ...      ...
394       2  0.999286    world
156       1  0.999702    worth
1165      5  0.999108     wrap
14        1  0.999855     year
341       4  0.999814   youtub

[342 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])

# BoltEV

In [24]:
data_BoltEV = pd.read_csv("BoltEV_Desc_Count.csv")

In [25]:
comments_BoltEV = data_BoltEV[data_BoltEV['body'].notna()]
comments_BoltEV = comments_BoltEV.reset_index()
comments_BoltEV = comments_BoltEV.drop(columns=['index'])

In [26]:
df = comments_BoltEV.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [27]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['[deleted]']


 tokenized and lemmatized document: 
['delet']


In [28]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 advic
1 base
2 bolt
3 charg
4 costco
5 deal
6 drive
7 edit
8 equival
9 fast
10 fee


In [29]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [30]:
lda_model_BoltEV = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model_BoltEV.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"brake" + 0.013*"like" + 0.013*"turn" + 0.012*"http" + 0.010*"light" + 0.010*"time" + 0.008*"work" + 0.008*"auto" + 0.007*"think" + 0.007*"issu"
Topic: 1 
Words: 0.044*"charg" + 0.038*"batteri" + 0.029*"mile" + 0.022*"drive" + 0.019*"rang" + 0.015*"time" + 0.011*"trip" + 0.011*"bolt" + 0.010*"heat" + 0.009*"cold"
Topic: 2 
Words: 0.050*"bolt" + 0.025*"drive" + 0.022*"tire" + 0.018*"like" + 0.016*"seat" + 0.015*"model" + 0.012*"volt" + 0.011*"year" + 0.010*"wheel" + 0.010*"love"
Topic: 3 
Words: 0.037*"charg" + 0.033*"charger" + 0.016*"http" + 0.016*"bolt" + 0.013*"plug" + 0.012*"need" + 0.012*"station" + 0.011*"work" + 0.010*"tesla" + 0.009*"electr"
Topic: 4 
Words: 0.021*"thank" + 0.019*"bolt" + 0.018*"price" + 0.018*"dealer" + 0.014*"year" + 0.012*"credit" + 0.011*"good" + 0.011*"know" + 0.011*"month" + 0.011*"work"


In [31]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_BoltEV, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.025126 -0.202217       1        1  25.795959
2      0.106714  0.037020       2        1  19.869890
4      0.116408  0.132965       3        1  19.469572
3      0.040637 -0.060221       4        1  17.812901
0     -0.238634  0.092453       5        1  17.051674, topic_info=    Category         Freq     Term        Total  loglift  logprob
3    Default  5825.000000    charg  5825.000000  30.0000  30.0000
146  Default  3190.000000  batteri  3190.000000  29.0000  29.0000
661  Default  2105.000000  charger  2105.000000  28.0000  28.0000
6    Default  3526.000000    drive  3526.000000  27.0000  27.0000
2    Default  6241.000000     bolt  6241.000000  26.0000  26.0000
..       ...          ...      ...          ...      ...      ...
74    Topic5   348.819153    thing  1217.392334   0.5190  -5.0643
133   Topic5   413.469788    think  2230.255859   0.0836  -4.8943
84    Topic5   292.634979    start  1140.507812   0.4086  -5.2399
86    Topic5   289.024078     come  1138.817505   0.3977  -5.2523
170   Topic5   261.750793      tri   676.655273   0.8192  -5.3515

[354 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1019      1  0.992843   accur
1493      2  0.105763   adapt
1493      4  0.891938   adapt
755       2  0.990816    amaz
1088      4  0.974460  amazon
...     ...       ...     ...
23        5  0.191109    work
41        1  0.084098    year
41        2  0.374914    year
41        3  0.461725    year
41        4  0.079215    year

[607 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 4, 1])

# Leaf

In [40]:
data_Leaf = pd.read_csv("Leaf_Desc_Count.csv")

In [41]:
comments_Leaf = data_Leaf[data_Leaf['body'].notna()]
comments_Leaf = comments_Leaf.reset_index()
comments_Leaf = comments_Leaf.drop(columns=['index'])

In [43]:
df = comments_Leaf.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [35]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['the', 'suggestion', 'was', 'serious.', '', 'yes,', "it's", 'been', 'done--to', 'a', '2015.', '', 'however,', 'i', "don't", 'see', 'any', 'underbody', 'battery', 'covers,', 'so', "i'm", 'assuming', 'they', "didn't", 'find', 'a', 'way', 'to', 'make', 'them', 'fit.\n\nhttps://www.youtube.com/watch?v=h47rnjsd0ys\n\nit', 'is', 'a', 'different', 'physical', 'size,', 'but', "it's", 'absolutely', 'possible.', '', 'the', 'cost', 'of', 'the', 'procedure', 'would', 'be', 'the', 'cost', 'of', 'getting', 'a', 'salvage', 'pack', 'plus', 'around', '$2k', '($1k', 'for', 'labor', 'and', '$1k', 'for', 'the', 'canbus', 'adapter).\n\nthe', 'cost', '&', 'value', 'of', 'the', 'car', 'pre-modification', "isn't", 'super', 'relevant.', '', 'modding', 'and', 'hot-rodding', 'cars', "isn't", 'about', 'the', 'intrinsic', 'value', 'of', 'the', 'car', 'before', 'modification.', '', "it's", 'about', 'creating', 'something', 'unique,', 'special,', 'or', 'just', 'what', 'you', 'want.\n\nit', 'seem

In [36]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 acceler
1 accumul
2 addit
3 advis
4 appear
5 area
6 best
7 brake
8 cabl
9 canada
10 care


In [38]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [39]:
lda_model_Leaf = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model_Leaf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.048*"charg" + 0.042*"mile" + 0.027*"drive" + 0.026*"batteri" + 0.025*"leaf" + 0.025*"rang" + 0.014*"charger" + 0.011*"time" + 0.011*"work" + 0.011*"trip"
Topic: 1 
Words: 0.032*"batteri" + 0.030*"leaf" + 0.025*"year" + 0.021*"nissan" + 0.015*"dealer" + 0.015*"price" + 0.012*"replac" + 0.011*"model" + 0.011*"buy" + 0.011*"warranti"
Topic: 2 
Words: 0.044*"http" + 0.036*"leaf" + 0.035*"thank" + 0.026*"look" + 0.019*"nissan" + 0.016*"like" + 0.013*"know" + 0.013*"think" + 0.013*"good" + 0.012*"post"
Topic: 3 
Words: 0.023*"work" + 0.013*"leaf" + 0.013*"like" + 0.011*"need" + 0.011*"connect" + 0.009*"phone" + 0.009*"batteri" + 0.008*"time" + 0.007*"seat" + 0.007*"auto"
Topic: 4 
Words: 0.023*"charg" + 0.019*"batteri" + 0.017*"power" + 0.012*"brake" + 0.011*"tire" + 0.011*"heat" + 0.011*"plug" + 0.010*"time" + 0.010*"drive" + 0.009*"like"


In [44]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_Leaf, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.108433  0.141863       1        1  30.585840
1     -0.058941  0.101460       2        1  25.052351
4      0.170729 -0.011984       3        1  19.511007
3      0.042830 -0.222998       4        1  12.997951
2     -0.263050 -0.008342       5        1  11.852843, topic_info=    Category         Freq    Term        Total  loglift  logprob
188  Default  5329.000000    mile  5329.000000  30.0000  30.0000
359  Default  7914.000000   charg  7914.000000  29.0000  29.0000
34   Default  2275.000000    http  2275.000000  28.0000  28.0000
138  Default  1727.000000   thank  1727.000000  27.0000  27.0000
207  Default  3227.000000    rang  3227.000000  26.0000  26.0000
..       ...          ...     ...          ...      ...      ...
223   Topic5   256.179474   great   936.773193   0.8360  -5.2531
816   Topic5   232.928726   sound   638.617554   1.1240  -5.3482
59    Topic5   257.379639  vehicl  1158.083008   0.6286  -5.2484
189   Topic5   272.599457   model  1842.454102   0.2218  -5.1910
173   Topic5   228.953278    make   836.534058   0.8369  -5.3654

[344 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         3  0.996666   acceler
946       4  0.999054    amazon
1302      5  0.984782  american
1187      3  0.998210       amp
1026      4  0.998407   android
...     ...       ...       ...
521       4  0.016213      yeah
521       5  0.453970      yeah
92        1  0.304724      year
92        2  0.695061      year
131       5  0.995068    youtub

[588 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3])

# Mercedes

In [45]:
data_mercedes = pd.read_csv("Mercedes_Desc_Count.csv")

In [46]:
comments_mercedes = data_mercedes[data_mercedes['body'].notna()]
comments_mercedes = comments_mercedes.reset_index()
comments_mercedes = comments_mercedes.drop(columns=['index'])

In [47]:
df = comments_mercedes.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [48]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['but', 'service', 'at', 'mercedes', 'are', 'so', 'expensive,', 'it', 'is', 'like', '750', 'for', 'a', 'service', 'each', 'year.', 'where', 'as', 'oil', 'change', 'cost', 'me', '35', 'dollars,', 'to', 'do', 'it', 'myself']


 tokenized and lemmatized document: 
['servic', 'merced', 'expens', 'like', 'servic', 'year', 'chang', 'cost', 'dollar']


In [49]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 appreci
1 authent
2 help
3 hood
4 order
5 ornament
6 rip
7 wonder
8 merced
9 model
10 year


In [50]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [51]:
lda_model_mercedes = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model_mercedes.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.057*"delet" + 0.024*"drive" + 0.023*"nice" + 0.022*"peopl" + 0.019*"know" + 0.019*"car" + 0.017*"yeah" + 0.013*"think" + 0.011*"mean" + 0.010*"engin"
Topic: 1 
Words: 0.077*"http" + 0.042*"merced" + 0.032*"benz" + 0.026*"post" + 0.022*"reddit" + 0.021*"comment" + 0.017*"messag" + 0.016*"remov" + 0.014*"imgur" + 0.011*"updat"
Topic: 2 
Words: 0.018*"year" + 0.017*"thank" + 0.012*"engin" + 0.011*"good" + 0.011*"like" + 0.010*"mile" + 0.010*"issu" + 0.009*"time" + 0.009*"replac" + 0.009*"go"
Topic: 3 
Words: 0.018*"beauti" + 0.017*"work" + 0.015*"drive" + 0.013*"seat" + 0.012*"start" + 0.011*"door" + 0.011*"turn" + 0.010*"park" + 0.010*"light" + 0.010*"need"
Topic: 4 
Words: 0.063*"look" + 0.044*"like" + 0.033*"class" + 0.026*"love" + 0.015*"think" + 0.015*"wheel" + 0.015*"model" + 0.014*"better" + 0.012*"interior" + 0.012*"black"


In [52]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_mercedes, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.073593  0.024012       1        1  36.824505
4     -0.126964 -0.046069       2        1  23.392630
3     -0.081803  0.284652       3        1  16.667994
0     -0.120604 -0.238784       4        1  13.402769
1      0.402964 -0.023811       5        1   9.712101, topic_info=     Category         Freq    Term        Total  loglift  logprob
59    Default  6392.000000    look  6392.000000  30.0000  30.0000
167   Default  2964.000000    http  2964.000000  29.0000  29.0000
153   Default  3026.000000   delet  3026.000000  28.0000  28.0000
117   Default  3112.000000   class  3112.000000  27.0000  27.0000
8     Default  4009.000000  merced  4009.000000  26.0000  26.0000
...       ...          ...     ...          ...      ...      ...
743    Topic5   231.509460   share   284.346497   2.1262  -5.1133
415    Topic5   356.749725  pictur   614.549744   1.7879  -4.6809
1656   Topic5   219.918076    send   264.095520   2.1487  -5.1647
658    Topic5   314.081085    edit   536.006226   1.7973  -4.8083
164    Topic5   338.277893   check  1067.517578   1.1826  -4.7341

[298 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1446      3  0.992337   activ
43        1  0.201130  actual
43        2  0.282670  actual
43        3  0.115967  actual
43        4  0.360585  actual
...     ...       ...     ...
145       2  0.155814    yeah
145       4  0.843608    yeah
10        1  0.911286    year
10        2  0.088679    year
936       5  0.998544  youtub

[419 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 2])

# Tesla

In [53]:
data_tesla = pd.read_csv("Tesla_Desc_Count.csv")

C:\Users\keert\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
comments_tesla = data_tesla[data_tesla['body'].notna()]
comments_tesla = comments_tesla.reset_index()
comments_tesla = comments_tesla.drop(columns=['index'])

In [55]:
df = comments_tesla.copy()

#change any int/float to str
df['body'] = df['body'].apply(str)

# Convert the reviews to lowercase
df['body'] = df['body'].map(lambda x: x.lower())

In [56]:
#Checking with sample review
doc_sample = df.loc[500,'body']
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['me', 'too,', 'completely', 'broken', 'now.']


 tokenized and lemmatized document: 
['complet', 'break']


In [57]:
df['comments_processed'] = pd.DataFrame(df['body'].map(preprocess))

dictionary = gensim.corpora.Dictionary(df['comments_processed'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 act
1 adult
2 believ
3 break
4 caus
5 combat
6 come
7 damag
8 folk
9 gener
10 go


In [58]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in df['comments_processed']]

In [62]:
lda_model_tesla = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=20, workers=2)
for idx, topic in lda_model_tesla.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.091*"http" + 0.038*"comment" + 0.029*"post" + 0.028*"teslamotor" + 0.028*"thank" + 0.018*"reddit" + 0.017*"tesla" + 0.017*"usag" + 0.015*"stub" + 0.014*"thread"
Topic: 1 
Words: 0.035*"charg" + 0.024*"batteri" + 0.017*"mile" + 0.016*"rang" + 0.016*"supercharg" + 0.015*"delet" + 0.011*"power" + 0.011*"charger" + 0.010*"drive" + 0.009*"need"
Topic: 2 
Words: 0.034*"tesla" + 0.014*"peopl" + 0.012*"price" + 0.012*"year" + 0.011*"car" + 0.010*"think" + 0.010*"like" + 0.009*"model" + 0.008*"say" + 0.007*"compani"
Topic: 3 
Words: 0.017*"drive" + 0.011*"like" + 0.011*"time" + 0.009*"work" + 0.009*"updat" + 0.009*"turn" + 0.008*"think" + 0.008*"lane" + 0.007*"park" + 0.007*"chang"
Topic: 4 
Words: 0.029*"model" + 0.024*"like" + 0.020*"look" + 0.013*"tesla" + 0.011*"think" + 0.009*"servic" + 0.009*"know" + 0.008*"week" + 0.008*"deliveri" + 0.008*"wait"


In [63]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tesla, bow_corpus, dictionary)
vis

C:\Users\keert\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.093676  0.052489       1        1  31.376925
3     -0.103709  0.036276       2        1  22.589537
4     -0.118624  0.165585       3        1  20.888966
1     -0.102143 -0.258850       4        1  16.986521
0      0.418151  0.004500       5        1   8.158052, topic_info=    Category           Freq     Term          Total  loglift  logprob
605  Default  109505.000000     http  109505.000000  30.0000  30.0000
445  Default   87329.000000    charg   87329.000000  29.0000  29.0000
285  Default   45193.000000  comment   45193.000000  28.0000  28.0000
752  Default   61696.000000  batteri   61696.000000  27.0000  27.0000
404  Default   35175.000000     post   35175.000000  26.0000  26.0000
..       ...            ...      ...            ...      ...      ...
29    Topic5   20909.333984    tesla  253717.781250   0.0101  -4.0507
658   Topic5    9143.637695      see   31085.218750   1.2825  -4.8778
399   Topic5    9556.194336   vehicl   50497.867188   0.8414  -4.8337
250   Topic5    7061.947266     read   17910.779297   1.5755  -5.1361
203   Topic5    7533.050293    model  155468.546875  -0.5210  -5.0716

[312 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
790       5  0.999882  acronym
268       1  0.999909   afford
1767      3  0.999827  appoint
100       1  0.226607     auto
100       2  0.773355     auto
...     ...       ...      ...
402       1  0.727969     year
402       3  0.172850     year
402       4  0.099180     year
1061      5  0.999857    youtu
1383      5  0.999903   youtub

[460 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 2, 1])

In [65]:
df

,author_id,author_name,body,downs,id,isPost,parent_id,post_id,time,title,ups,url,descendant_count,comments_processed
0,t2_3mano45q,mx_thunder,i could never understand why there were so man...,0,t1_fcm8m2e,False,t3_ehxuzw,t3_ehxuzw,1.577767e+09,WSJ full page ad from today. Is there any trut...,1.0,NaN,0,"[understand, video, show, peopl, mainli, grow,..."
1,t2_1e2pyrns,qrsl_,title pretty much sums it up. when driving in ...,0,t3_ehxewz,True,NaN,t3_ehxewz,1.577764e+09,How to switch from autopilot back to cruise co...,11.0,https://www.reddit.com/r/teslamotors/comments/...,43,"[titl, pretti, sum, drive, freeway, prefer, lo..."
2,t2_12c5pq,planko13,if you fully engage the blinker (so it will bl...,0,t1_fcm59bk,False,t3_ehxewz,t3_ehxewz,1.577765e+09,How to switch from autopilot back to cruise co...,44.0,NaN,13,"[fulli, engag, blinker, blink, forev, stop, fi..."
3,t2_jlzqv,aspec818,sounds like you have basic autopilot. the easi...,0,t1_fcmjhbl,False,t3_ehxewz,t3_ehxewz,1.577778e+09,How to switch from autopilot back to cruise co...,14.0,NaN,1,"[sound, like, basic, autopilot, easiest, switc..."
4,t2_6smdg,110110,i just push the drive stalk upwards in my 3,0,t1_fcm7pd8,False,t3_ehxewz,t3_ehxewz,1.577766e+09,How to switch from autopilot back to cruise co...,15.0,NaN,1,"[push, drive, stalk, upward]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142978,t2_kggns,sodakas,makes perfect sense; i certainly expect this t...,0,t1_ed0xnqk,False,t1_ed0wwhx,t3_abg2jr,1.546368e+09,Opened my trunk cover for the first time and f...,1.0,NaN,1,"[make, perfect, sens, certainli, expect, super..."
1142979,t2_80g4j,The_Great_Squijibo,waited 7 hours in front of montreal store in t...,0,t1_ed0itrn,False,t1_ed03gg4,t3_abg2jr,1.546355e+09,Opened my trunk cover for the first time and f...,3.0,NaN,0,"[wait, hour, montreal, store, cold, dark, receiv]"
1142980,t2_7ewl5,pvtcookie,they sure did! they verified my delivery date ...,0,t1_ed1gd7f,False,t1_ed10zit,t3_abg2jr,1.546381e+09,Opened my trunk cover for the first time and f...,0.0,NaN,0,"[sure, verifi, deliveri, date, info, match, fi..."
1142981,t2_as01w2,izybit,make that every 1-2 months and you'll get one.,0,t1_ed125xv,False,t1_ed0xnqk,t3_abg2jr,1.546371e+09,Opened my trunk cover for the first time and f...,1.0,NaN,0,[month]


In [73]:
texts = df['comments_processed']
id2word = gensim.corpora.Dictionary(texts.str())
corpus = [id2word.doc2bow(text) for text in texts]

#Compute Perplexity -- the lower the better
print('\nPerplexity: ', lda_model_tesla.log_perplexity(bow_corpus))

from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model_tesla, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

TypeError: 'StringMethods' object is not callable

In [75]:
df['comments_processed']

0          [understand, video, show, peopl, mainli, grow,...
1          [titl, pretti, sum, drive, freeway, prefer, lo...
2          [fulli, engag, blinker, blink, forev, stop, fi...
3          [sound, like, basic, autopilot, easiest, switc...
4                               [push, drive, stalk, upward]
                                 ...                        
1142978    [make, perfect, sens, certainli, expect, super...
1142979    [wait, hour, montreal, store, cold, dark, receiv]
1142980    [sure, verifi, deliveri, date, info, match, fi...
1142981                                              [month]
1142982                                              [remov]
Name: comments_processed, Length: 1142983, dtype: object